<a href="https://colab.research.google.com/github/bxck75/A1_Colabs/blob/master/nicedcgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##FirstGan

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os,sys,shutil
def mdir(path):
    state = False
    if not os.path.exists(path):
        os.makedirs(path)
        state = True

    return state

# Python program to shutil.copy()
def scp(source, destination):
    state = False
    try:
        shutil.copy(source, destination)
        print("File copied successfully.")
        state = True
    # If source and destination are same
    except shutil.SameFileError:
        print("Source and destination represents the same file.")
    # If there is any permission issue
    except PermissionError:
        print("Permission denied.")
    # For other errors
    except:
        print("Error occurred while copying file.")

    return state


In [ ]:
datasetname= 'garbage'
images_drive_dir = '"/content/drive/MyDrive/Colab Notebooks/datasets/garbage"'
mdir(datasetname)
images_colab_dir = '/content/'+datasetname+'/input_images'

if mdir(images_colab_dir):
    print('copy files')

!cp -rf {images_drive_dir} {images_colab_dir}
#scp(images_drive_dir, images_colab_dir)

#url = "https://drive.google.com/uc?id=1O7m1010EJjLE5QxLZiM9Fpjs7Oj6e684"
#output = datasetname + "/data.zip"
#gdown.download(url, output, quiet=True)

#with ZipFile("/content/drive/MyDrive/garbagepailkids.zip", "r") as zipobj:
    #zipobj.extractall(datasetname)


In [ ]:
image_folder = '/content/'+datasetname+'/input_images/'+ datasetname 

In [ ]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import gdown
from zipfile import ZipFile

In [ ]:
res = 128
dataset = keras.utils.image_dataset_from_directory(
    image_folder, label_mode=None, image_size=(res, res), batch_size=4
)
dataset = dataset.map(lambda x: x / 255.0)
for x in dataset:
    plt.axis("off")
    plt.imshow((x.numpy() * 127).astype("int32")[0])
    break

latent_dim = 128

In [ ]:
discriminator = keras.Sequential(
    [
        keras.Input(shape=(64, 64, 3)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)

discriminator.summary()


In [ ]:
generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(8 * 8 * 128),
        layers.Reshape((8, 8, 128)),
        layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
#        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
#        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
    ],
    name="generator",
)

generator.summary()


In [ ]:
generator.save("/content/garbage/output_model/Generator_{res}.h5")
discriminator.save("/content/garbage/output_model/Discriminator_{res}.h5")

In [ ]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.generator.save("/content/garbage/output_model/Generator_{res}.h5")
        self.discriminator.save("/content/garbage/output_model/Discriminator_{res}.h5")

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }



In [ ]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=10, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim
    
    def on_epoch_begin(self,epoch, logs=None):



    def check_retrain_state(self,weightfile):
        if os.path.exists(weightfile):
            return True
        else:
            return False

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = keras.utils.array_to_img(generated_images[i])
            img.save("/content/garbage/out_images/generated_img_%03d_%d.png" % (epoch, i))
            
        #s#elf.model.generator.save_weights('/content/gen_Weights_{epoch:02d}.h5')
        #print("gen weights saved")
        #self.model.discriminator.save_weights('/content/dis_Weights_{epoch:02d}.h5')
        #print("dis weights saved")



In [ ]:
cycles = 2
for cycle in range(0,cycles):

    my_callbacks = [GANMonitor(num_img=10, latent_dim=latent_dim),]

    epochs = 3    # In practice, use ~100 epochs
    gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)

    if os.path.exists('/content/garbage/output_model/gen_Weights'+str(cycle)+'.h5'):
        gan.generator.load_weights('/content/garbage/output_model/gen_Weights'+str(cycle)+'.h5')
        print('gen loaded')
    if os.path.exists('/content/garbage/output_model/dis_Weights'+str(cycle)+'.h5'):
        gan.discriminator.load_weights('/content/garbage/output_model/dis_Weights'+str(cycle)+'.h5')
        print('dis loaded')

    gan.compile(
        d_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
        loss_fn=keras.losses.BinaryCrossentropy(),
    )

    gan.fit(
        dataset, epochs=epochs, callbacks=my_callbacks
    )

    gan.generator.save_weights('/content/garbage/output_model/gen_Weights'+str(cycle)+'.h5')
    gan.discriminator.save_weights('/content/garbage/output_model/dis_Weights'+str(cycle)+'.h5')
        


# Finalize the model in \x_model folder,
# in saveModel format: 


##KerasClassify

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 320, 240

train_data_dir = '/content/garbage/input_images/garbage'
validation_data_dir = '/content/garbage/input_images/garbage/llery_2k13mini'
nb_train_samples = 3000  #total
nb_validation_samples = 10  # total
epochs = 16
batch_size = 10

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=5)

model.save('model.h5')

In [ ]:
from keras.models import load_model
import cv2
import numpy as np

model = load_model('model.h5')

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/datasets/garbage/llery_2k13mini/13mini_195a.jpg')
img = cv2.resize(img,(320,240))
img = np.reshape(img,[1,320,240,3])

classes = model.predict(img)

print(classes)

In [ ]:
def get_model():
    # Create a simple model.
    inputs = keras.Input(shape=(32,))
    outputs = keras.layers.Dense(1)(inputs)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss="mean_squared_error")
    return model


model = get_model()

# Train the model.
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

# Calling `save('my_model')` creates a SavedModel folder `my_model`.
model.save("my_model")

# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("my_model")

# Let's check:
np.testing.assert_allclose(
    model.predict(test_input), reconstructed_model.predict(test_input)
)

# The reconstructed model is already compiled and has retained the optimizer
# state, so training can resume:
reconstructed_model.fit(test_input, test_target)


In [ ]:
!rm -r /content/garbage/output_model/gen_Weights*